In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
#config_file = "configs/vgg_ssd300_voc0712_tdt4265_server.yaml"
#config_file = "configs/mnist_tdt4265_server_experimental.yaml"
config_file = "configs/train_rdd2020_server.yaml"
# config_file = "configs/train_rdd2020_server_experimental.yaml"
cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-22 16:11:45,216 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-22 16:11:45,218 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'res_net'
        PRETRAINED: True
        OUT_CHANNELS: [128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
INPUT:
    IMAGE_SIZE: [300, 300]
DATASETS:
    TRAIN: ("rdd2020_train",)
    TEST: ("rdd2020_val", )
SOLVER:
    MAX_ITER: 120000
    GAMMA: 0.1
    BATCH_SIZE: 8 # NOTE CHANGED BATCH SIZE FROM 16 TO 8
    LR: 1e-3
    WEIGHT_DECAY: 5e-3 # INCREASED WEIGHT DECAY
OUTPUT_DIR: 'outputs/rdd2020_weight_decay_5e3_batch_size_8_data_augmentation'
DATASET_DIR: "datasets"
MODEL_SAVE_STEP: 2000
EVAL_STEP: 10000 

2021-04-22 16:11:45,219 SSD INFO: Running with config:
DATASETS:
  TEST: ('rdd2020_val',)
  TRAIN: ('rdd2020_train',)
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 4
  PIN_MEMORY: True
  USE_ALBUMENTATION: False
EVAL_STEP: 10000
INPUT:
  IMAGE_SIZE: [300, 300]
  PIXEL_MEAN: [123.675, 116.28,

In [ ]:
!pwd 
model = start_train(cfg)

/notebooks/TDT4265-Project/assignment4/SSD
Detector initialized. Total Number of params:  21.62M
Backbone number of parameters: 20.84M
SSD Head number of parameters: 788.2K
SSDDetector(
  (backbone): ResNet18(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

  0%|          | 0/366 [00:00<?, ?it/s]/notebooks/TDT4265-Project/assignment4/SSD/ssd/modeling/box_head/inference.py:43: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:760.)
  indices = torch.nonzero(scores > self.cfg.TEST.CONFIDENCE_THRESHOLD).squeeze(1)
100%|██████████| 366/366 [06:16<00:00,  1.08s/it]


2021-04-22 16:30:09,453 SSD.inference INFO: mAP: 0.1892
D00             : 0.1676
D10             : 0.0615
D20             : 0.3601
D40             : 0.1674

2021-04-22 16:30:10,172 SSD.trainer INFO: iter: 010010, lr: 0.00100, total_loss: 4.481 (5.049), reg_loss: 1.651 (1.970), cls_loss: 2.830 (3.080), time: 37.970 (0.109), eta: 3:19:47, mem: 1561M
2021-04-22 16:30:10,879 SSD.trainer INFO: iter: 010020, lr: 0.00100, total_loss: 4.600 (5.049), reg_loss: 1.754 (1.970), cls_loss: 2.845 (3.079), time: 0.071 (0.109), eta: 3:19:42, mem: 1561M
2021-04-22 16:30:11,622 SSD.trainer INFO: iter: 010030, lr: 0.00100, total_loss: 4.681 (5.049), reg_loss: 1.766 (1.970), cls_loss: 2.916 (3.079), time: 0.074 (0.109), eta: 3:19:37, mem: 1561M
2021-04-22 16:30:12,315 SSD.trainer INFO: iter: 010040, lr: 0.00100, total_loss: 4.342 (5.048), reg_loss: 1.582 (1.969), cls_loss: 2.760 (3.079), time: 0.069 (0.109), eta: 3:19:32, mem: 1561M
2021-04-22 16:30:13,029 SSD.trainer INFO: iter: 010050, lr: 0.00100, total

In [1]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

NameError: name 'logger' is not defined